In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image, ImageFile
from transformers import DistilBertTokenizer, DistilBertModel, ViTImageProcessor, ViTModel
import torch.nn as nn
import os
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score

In [2]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.backends.cudnn.benchmark = True

In [4]:
device

device(type='cuda')

In [5]:
class MemeDataset(Dataset):
    def __init__(self, labels_path, image_dir, text_max_length=128):
        self.labels_df = pd.read_csv(labels_path)
        self.image_dir = image_dir
        
        # Data validation
        self.labels_df['text_corrected'] = self.labels_df['text_corrected'].astype(str)
        
        self.tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
        self.text_max_length = text_max_length
        
        self.image_processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
        
        self.label_maps = {
            'sentiment': ['very_negative', 'negative', 'neutral', 'positive', 'very_positive'],
            'humor': ['not_funny', 'funny', 'very_funny', 'hilarious'],
            'sarcasm': ['not_sarcastic', 'general', 'twisted_meaning', 'very_twisted'],
            'offensive': ['not_offensive', 'slight', 'very_offensive', 'hateful_offensive'],
            'motivational': ['not_motivational', 'motivational']
        }

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        row = self.labels_df.iloc[idx]
        
        # Text processing
        text = str(row['text_corrected'])
        inputs = self.tokenizer(
            text,
            max_length=self.text_max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )
        
        # Image processing
        img_path = os.path.join(self.image_dir, row['image_name'])
        image = Image.open(img_path).convert('RGB')
        pixel_values = self.image_processor(images=image, return_tensors="pt").pixel_values
        
        # Label encoding
        labels = {
            'sentiment': torch.tensor(self.label_maps['sentiment'].index(row['overall_sentiment']), dtype=torch.long),
            'humor': torch.tensor(self.label_maps['humor'].index(row['humour']), dtype=torch.long),
            'sarcasm': torch.tensor(self.label_maps['sarcasm'].index(row['sarcasm']), dtype=torch.long),
            'offensive': torch.tensor(self.label_maps['offensive'].index(row['offensive']), dtype=torch.long),
            'motivational': torch.tensor(1 if row['motivational'] == 'motivational' else 0, dtype=torch.long)
        }
        
        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'pixel_values': pixel_values.squeeze(),
            'labels': labels
        }

In [6]:
class MultimodalModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Text encoder
        self.text_model = DistilBertModel.from_pretrained('distilbert-base-uncased')
        
        # Image encoder
        self.image_model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
        
        # Multimodal fusion
        self.fusion = nn.Sequential(
            nn.Linear(768*2, 512),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        
        # Classification heads
        self.classifier = nn.ModuleDict({
            'sentiment': nn.Linear(512, 5),
            'humor': nn.Linear(512, 4),
            'sarcasm': nn.Linear(512, 4),
            'offensive': nn.Linear(512, 4),
            'motivational': nn.Linear(512, 2)
        })

    def forward(self, input_ids, attention_mask, pixel_values):
        text_out = self.text_model(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:,0,:]
        image_out = self.image_model(pixel_values=pixel_values).last_hidden_state[:,0,:]
        
        fused = torch.cat([text_out, image_out], dim=1)
        fused = self.fusion(fused)
        
        return {task: self.classifier[task](fused) for task in self.classifier}

In [7]:
def train_model(model, dataloader, optimizer, criterion, epochs=2):
    model.train()
    
    for epoch in range(epochs):
        total_loss = 0
        progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}")
        
        for batch in progress_bar:
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
            labels = {k: v.to(device) for k, v in batch['labels'].items()}
            
            optimizer.zero_grad()
            outputs = model(**inputs)
            
            loss = sum(criterion[task](outputs[task], labels[task]) for task in outputs)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            progress_bar.set_postfix({'loss': f"{loss.item():.4f}"})
        
        print(f"Epoch {epoch+1} Avg Loss: {total_loss/len(dataloader):.4f}")
    
    return model

In [8]:
def evaluate_model(model, dataloader):
    model.eval()
    all_preds, all_labels = {task: [] for task in criterion}, {task: [] for task in criterion}

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
            labels = {k: v.cpu().numpy() for k, v in batch['labels'].items()}

            outputs = model(**inputs)
            preds = {task: torch.argmax(outputs[task], dim=1).cpu().numpy() for task in outputs}

            for task in preds:
                all_preds[task].extend(preds[task])
                all_labels[task].extend(labels[task])

    # Calculate metrics
    metrics = {}
    for task in criterion:
        metrics[task] = {
            "Accuracy": accuracy_score(all_labels[task], all_preds[task]),
            "F1 Score": f1_score(all_labels[task], all_preds[task], average="macro"),
        }

    return metrics

In [9]:
# dataset = MemeDataset(labels_path='D:/Multimodal Sentiment Analysis/Major Assignment 3/Multimodal_dataset_assignment3/Multimodal_dataset_assignment3/labels.csv', image_dir='D:/Multimodal Sentiment Analysis/Major Assignment 3/Multimodal_dataset_assignment3/Multimodal_dataset_assignment3/images')
# dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

from sklearn.model_selection import train_test_split

# Load dataset
labels_path = 'C:/Users/rouna/OneDrive/Desktop/Shashi Project/Multimodal_dataset_assignment3/Multimodal_dataset_assignment3/labels.csv'
image_dir = 'C:/Users/rouna/OneDrive/Desktop/Shashi Project/Multimodal_dataset_assignment3/Multimodal_dataset_assignment3/images'

full_df = pd.read_csv(labels_path)

# Split data into train and validation sets
train_df, val_df = train_test_split(full_df, test_size=0.2, random_state=42, stratify=full_df['overall_sentiment'])

# Save split datasets to new CSV files
train_df.to_csv("train_labels.csv", index=False)
val_df.to_csv("val_labels.csv", index=False)

# Create datasets
train_dataset = MemeDataset(labels_path='train_labels.csv', image_dir=image_dir)
val_dataset = MemeDataset(labels_path='val_labels.csv', image_dir=image_dir)

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)


In [10]:
model = MultimodalModel().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

In [11]:
criterion = {
    task: nn.CrossEntropyLoss()
    for task in ['sentiment', 'humor', 'sarcasm', 'offensive', 'motivational']
}

In [12]:
# trained_model = train_model(model, dataloader, optimizer, criterion)
trained_model = train_model(model, train_dataloader, optimizer, criterion)

Epoch 1/2:  46%|████▌     | 160/350 [02:06<02:15,  1.40it/s, loss=5.6942]c:\Users\rouna\AppData\Local\Programs\Python\Python313\Lib\site-packages\PIL\Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Epoch 1/2: 100%|██████████| 350/350 [04:21<00:00,  1.34it/s, loss=5.7952]


Epoch 1 Avg Loss: 5.6566


Epoch 2/2: 100%|██████████| 350/350 [04:27<00:00,  1.31it/s, loss=5.7108]

Epoch 2 Avg Loss: 5.5605


In [13]:
torch.save(trained_model.state_dict(), 'memotion_model.pth')
print("Model saved successfully!")

Model saved successfully!


In [14]:
# Load model for evaluation
model.load_state_dict(torch.load("memotion_model.pth", map_location=device))
model.to(device)

MultimodalModel(
  (text_model): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): 

In [15]:
# metrics = evaluate_model(model, test_dataloader)
metrics = evaluate_model(model, val_dataloader)

Evaluating: 100%|██████████| 88/88 [00:39<00:00,  2.25it/s]


In [16]:
for task, scores in metrics.items():
    print(f"\nTask: {task}")
    print(f"  Accuracy: {scores['Accuracy']:.4f}")
    print(f"  Macro F1 Score: {scores['F1 Score']:.4f}")


Task: sentiment
  Accuracy: 0.4475
  Macro F1 Score: 0.1237

Task: humor
  Accuracy: 0.3495
  Macro F1 Score: 0.1400

Task: sarcasm
  Accuracy: 0.5118
  Macro F1 Score: 0.1693

Task: offensive
  Accuracy: 0.3896
  Macro F1 Score: 0.1891

Task: motivational
  Accuracy: 0.6576
  Macro F1 Score: 0.3967
